In [1]:
import pandas as pd
%matplotlib inline
import matplotlib
matplotlib.style.use('ggplot')

As you may have seen, I'm back through 2008 on the Big 3 (Governor's Ball has only been multiple days for the last 4 years, so I stopped after that). I'd like to do a graph over time of the % of unique bands for each festival. I don't know if there's a quick code way to do that, but I can also just count off the spreadsheet. I'm going to go back to 2005, when Lolla started as a destination fest.

In [19]:
years = range(2005,2017)
festivals = ['COACHELLA', 'GOVBALL', 'BONNAROO', 'LOLLA']

In [22]:
def load_df_year(year):
    """Given a year, return the cleaned df"""
    # Data clean and setup
    # df2016.drop([0,1], inplace=True)
    # df2016.drop(['AVG RANK','# OF FESTS', 'PWR RANKING', 'Unnamed: 4'], axis=1)
    df = pd.read_excel('/Users/hunterowens/Downloads/Festivals.xlsx', sheet=str(year))
    df.set_index('ARTIST', inplace=True)

In [23]:
[load_df_year(year) for year in years]

[None, None, None, None, None, None, None, None, None, None, None, None]

In [20]:
def compute_percentage_of_lineup_in_other_festivals(lineup, df): 
    """Given a lineup from d[FESTIVAL_NAME], tell me what percentage is in another festival """
    sum_lineup = df[lineup].count()
    all_artists_from_other_fests = df.drop(lineup, axis=1).dropna(how='all').index.get_values()
    fest_lineup_list = df[lineup].dropna().index.get_values()
    common_artists_sum = len(set(all_artists_from_other_fests).intersection(fest_lineup_list))
    percentage = common_artists_sum / sum_lineup
    return percentage